In [1]:
%pip install pydub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence

# Load the audio file
audio = AudioSegment.from_file("Backward.wav")

# Parameters
min_silence_len = 500  # Minimum length of silence to consider for splitting (ms)
silence_thresh = audio.dBFS - 14  # Silence threshold (in dBFS)
padding_duration = 1000  # Duration of padding to add before and after the word (ms)

def trim_leading_trailing_silence(segment, silence_thresh, max_silence_len):
    # Detect leading and trailing silence
    silent_chunks = detect_silence(segment, min_silence_len=100, silence_thresh=silence_thresh)
    
    if silent_chunks:
        # Assume the silence detected is before and after the word
        leading_silence_end = silent_chunks[0][1] if silent_chunks[0][0] == 0 else 0
        trailing_silence_start = silent_chunks[-1][0] if silent_chunks[-1][1] == len(segment) else len(segment)
        
        # Trim only the leading and trailing silence (up to max_silence_len)
        start = leading_silence_end
        end = trailing_silence_start

        if end - start < len(segment):
            # Ensure trimming is within bounds and retains the word
            start = max(0, start - max_silence_len)
            end = min(len(segment), end + max_silence_len)
            return segment[start:end]
    return segment

# Split audio based on silence
segments = split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

# Define the word to use in filenames
word = "Backward"

# Process each segment
for i, segment in enumerate(segments):
    # Remove leading and trailing silence
    trimmed_segment = trim_leading_trailing_silence(segment, silence_thresh, padding_duration)

    # Add padding to the beginning and end of each segment
    start_padding = AudioSegment.silent(duration=padding_duration)
    end_padding = AudioSegment.silent(duration=padding_duration)
    final_segment = start_padding + trimmed_segment + end_padding

    # Generate the filename
    filename = f"{word}_{i+1:02d}.wav"
    
    # Export the final segment
    final_segment.export(filename, format="wav")

print("Processing complete!")


Processing complete!


: 